# CS7290 Causal Modeling in Machine Learning: Homework 4
Sarthak Kothari, Manthan Shah, Siddhesh Acharekar

## 1. Necessity and Sufficiency (8 points)

### 1.1 Probability of Neccessity and Sufficiency
Consider the following data comparing purchases on an e-commerce website with high and low exposure to promotions.

| Purchase | Promo Exposure High X=1 | Promo Exposure Low X=0 |
| -------- | ----------------------- | ---------------------- |
| Yes (Y=1)| 930                     | 201                    |
| No (Y=0) | 81                      | 808                    |

| Purchase | Promo Exposure High X=1 | Promo Exposure Low X=0 | Probability |
| -------- | ----------------------- | ---------------------- | ----------- |
| Yes (Y=1)| 0.4604                  | 0.0995                 | $P(Y=1) = 0.5599$ |
| No (Y=0) | 0.0401                  | 808                    | $P(Y=0) = 0.4401$ |

|P(Y = y, X = x)| Conditional Probabilities |
| ------------- | ------------------------- |
|P(Y = 1 \| X = 0)|	0.1992071               |
|P(Y = 1 \| X = 1)|	0.9198813               |
|P(X = 1 \| Y = 0)|	0.0911136               |
|P(X = 1 \| Y = 1)|	0.8222812               |


Given these data, we wish to estimate the probabilities that high exposure to promotions was a necessary (or sufficient, or both) cause of purchase. We assume monotonicity – exposure to promotions didn’t cause anyone NOT to make a purchase.

We assume that purchases Y has the following simple disjunctive mechanism:

\begin{equation}
  M =
    \begin{cases}
      n_{x} \sim \text{BernoulliBool}(p=0.5)  \\
      n_{q} \sim \text{BernoulliBool}(p=0.9)  \\
      n_{y} \sim \text{BernoulliBool}(p=0.2)  \\
      x = n_{x} \\
      q = n_{q} \\
      y = (x \cap q) \cup n_{y} \\
    \end{cases}       
\end{equation}

Reformating the DAG to: 

We get the following state table:

|Row|X|Q|$n_y$|Y|$P(Y) = P(X)P(Q)P(n_y)$|
|-|-|-|-----|-|-|
|R1|0|0|0|0|0.03996|
|R2|0|0|1|1|0.00999|
|R3|0|1|0|0|0.35964|
|R4|0|1|1|1|0.08991|
|R5|1|0|0|0|0.04004|
|R6|1|0|1|1|0.01001|
|R7|1|1|0|1|0.36036|
|R8|1|1|1|1|0.09009|

#### 1.1.1 Calculate the probability of necessity:  $P(Y_0 = 0 | X=1, Y=1)$

For conditional $X = 1, Y = 1$

|Row|X|Q|$n_y$|Y|$P(Y) = P(X)P(Q)P(n_y)$|
|-|-|-|-----|-|-|
|R6|1|0|1|1|0.01001|
|R7|1|1|0|1|0.36036|
|R8|1|1|1|1|0.09009|

If $X = 0$, R7 changes to $Y = 0$
so $P(Y_0 = 0 | X=1, Y=1) = \frac{P(R7)}{P(R6) + P(R7) + P(R8)}$
$ = \frac{0.36036}{0.01001+0.36036+0.09009} = 0.7826$

#### 1.1.2 Calculate the probability of sufficiency:  $P(Y_1 = 1 | X=0, Y=0)$

For conditional $X = 0, Y = 0$

|Row|X|Q|$n_y$|Y|$P(Y) = P(X)P(Q)P(n_y)$|
|-|-|-|-----|-|-|
|R1|0|0|0|0|0.03996|
|R3|0|1|0|0|0.35964|

If $X = 1$, R3 changes to $Y = 1$
so $P(Y_1 = 1 | X=0, Y=0) = \frac{P(R3)}{P(R1) + P(R3)}$
$ = \frac{0.35964}{0.03996+0.35964} = 0.9$

### 1.1 Probability of Neccessity and Sufficiency, and Identifiability

Typically we don’t know the whole structural model. We would only have the statistical table above, Assume only the structural assignment for Y is known.

If X is exogenous and Y is monotonic relative to X, then the probabilities PN, PS, and PNS are all identifiable and are given by

\begin{align}
PNS &= P(Y=1|X=1)−P(Y=1|X=0) \\
PN &= \frac{PNS}{P(Y=1|X=1)} \\
PS &= \frac{PNS}{P(Y=0|X=0)}
\end{align}

#### 1.2.1 Find probability of neccessity and sufficiency in problem 1. 
$PNS = P(Y=1|X=1)−P(Y=1|X=0) = 0.9198813 - 0.1992071 = 0.7206742$

#### 1.2.2  Find PN and PS using just PNS and the conditional probabilities.
$PN = \frac{PNS}{P(Y=1|X=1)} = \frac{0.7206742}{0.9198813} = 0.7834426028662611$

$PS = \frac{PNS}{P(Y=0|X=0)} = \frac{0.7206742}{1 - 0.1992071} = 0.8999507862769512$

## 2. Mediation (12 points)

Suppose you are a developer for a freemium subscription content platform. Your company did an A/B test for a new feature, designed to increase conversions to a paid premium subscription. The variables here are $X∈\{0,1\}$ for whether or not a user was exposed to the feature, and $Y∈\{0,1\}$ for conversion.

Based on some analysis and domain knowledge, you come up with the following model.


\begin{equation}
  C =
    \begin{cases}
      X &= X_{X}  \\
      T &= 3*X + N_{T}  \\
      E &= 2*T + 8*X + N_{E}  \\
      Y &= I(E > 10 + N_{C}) 
    \end{cases}       
\end{equation}

Here T is “thrash”. Since the new feature changes the website’s UX, “thrash” quantifies the time and effort the user has to spend familiarizing themselves with the new UX. E is engagement. The model assumes that the more the user engages with the site the more likely they are to convert. I is an indicator function, it returns 1 if engagement (E) > 10, 0 otherwise. Though the A/B test ties to estimate the causal effect of X on Y, T and E are mediators of that effect. You want to know how much the feature drives conversions directly through engagement, and how much is just due to thrash (which might have negative consequences on other outcomes not explicitly included in this model).

$N_{X}$ comes from a fair-coin flip. All of the other noise variables are normal distributions with mean 0. However, for simplicity, we are going to assume noise variables all have a variance/standard deviation of 0. In other words, for our purposes you can assign a value of 0 to all the noise terms.

#### 2.1 Calculate the total effect of the feature on conversions.

$\text{TE}(X) = E[Y|do(X=1)] - E[Y|do(X=0)]$ <br>
When $X=0$, <br>
$T = 0$ <br>
$E = 0$ <br>
$Y = I(0>10) = 0$ <br>

When $X=1$ <br>
$T = 3$ <br>
$E = 6 + 8 = 14$ <br>
$Y = I(14>10) = 1$ <br>

So $\text{TE}(X) = E[Y|do(X=1)] - E[Y|do(X=0)] = 1-0 = 1$

#### 2.2. Calculate the natural indirect effect (NIE). NIE is the expected change in conversions, given no exposure to the feature, but set thrash (T) at the level it would take if one was exposed to the feature.

$\text{NIE} = E(Y_{T = 3} = 1|do(X = 0)) - E(Y_{T = 0} = 1|do(X = 0))$ <br>

For $E(Y_{T = 3} = 1|do(X = 0))$ <br>
$X = 0$
$T = 3$
$E = 6+0 = 6$ <br>
$Y = I(6>10) = 0$ <br>


For $E(Y_{T = 0} = 1|do(X = 0))$ so, <br>
$X = 0$ <br>
$T = 0$ <br>
$E = 0 + 0 = 0$ <br>
$Y = I(0>10) = 0$ <br>

$\text{NIE} = E(Y_{T = 3} = 1|do(X = 0)) - E(Y_{T = 0} = 1|do(X = 0)) = 0 - 0 = 0$

#### 2.3. Calculate the controlled direct effect (CDE) when thrash is 0. A CDE is the effect you get when holding a mediator at a fixed value.

$\text{CDE}(T=0) = E(Y = y |do(X = 1), do(T=0)) - E(Y = y |do(X = 0), do(T=0))$ <br>
For $X = 1$, $T = 0$ <br>
$E = 0 + 8 = 8$ <br>
$Y = I(8>10) = 0$ <br>

For $X = 0$, $T = 0$ <br>
$E = 0 + 0 = 0$ <br>
$Y = I(0>10) = 0$ <br>

$\text{CDE}(T=0) = E(Y = y |do(X = 1), do(T=0)) - E(Y = y |do(X = 0), do(T=0)) = 0 - 0 = 0$

#### 2.4. Compute the reverse natural indirect effect. Reverse NIE is expected change in conversions, given thrash (T) being fixed at feature exposure levels, but then setting X to 0. (2 points) (HINT: “change” means that going from 0 to 1 means the effect is a positive number, going from 1 to 0 means the effect has a negative number.)

$\text{Reverse NIE} = E(Y_{T = 0}|do(X = 1)) - E(Y_{T = 3}|do(X = 1))$ <br>

$\text{Reverse NIE} = 0 - 1 = -1$

#### 2.5. Compute the natural direct effect (ND) using the following formula: Total effect = NDE - reverse NIE. Explain what the implications of this is to the analysis of this feature? (1 point)

Total effect = NDE - reverse NIE <br>
NDE = Total effect + reverse NIE <br>
So, NDE = 1 + -1 = 0 <br>

**NDE is the expected increase in outcome as X changes from 0 to 1, while the mediator T is set to whatever value it would have attained prior the change, that is, under X = 0. So T shows the effect of a discriminatory variable indicating whatever the feature X is, if T is biased, the outcome Y will be biased too.**

#### 2.6. Discussion: If the noise variables were not degenerate (meaning the didn’t have non-zero variance), how would this have affected the calculations and the conclusion about the NDE? (1 point)

**If noise variable were not degenerate, then they would fluctuate from expeted value. This would influence Y in one off occurences but if we were to sample a large number of times then the expected values would be the same on average.**

#### 2.7. Suppose instead we used the following model.

\begin{equation}
  C =
    \begin{cases}
      X &= X_{X}  \\
      U &= N_{U} \\
      T &= 2*X + N_{T}  \\
      E &= 3*T + 7*X + N_{E}  \\
      Y &= I(g(E, U, N_{C}) > \epsilon) 
    \end{cases}       
\end{equation}

Here, U  is a vector of user-related features. g is a deep neural network that takes as input engagement (E) as well as these other user features and the noise term, and outputs a value. ϵ is a threshold. Describe in clear terms how this would change the above analysis, if at all. (2 points)

**The Neural network would learn and optimize for U and E but not for T. So whatever parameters that would get learnt for U and E to maximise outcome, the mediator T would still be able to influence the outcome because it is not controlled by the network. So the discriminatory effects, if any, would be affecting the outcome no matter what E and U are learnt.**

## 3. Effect of the treatment on the treated (9 points)

Suppose you work for a car-sharing service company like Uber. You find that many drivers are making decisions in ways that are sub-optimal for the drivers, often missing low-hanging fruit (e.g. picking up riders closer to where they live, or choosing to drive in areas that have less demand and yet more traffic than others). If the drivers made better decisions about when and where to drive, they could make more money with a similar amount of effort.

The company hires a statistical consulting company that samples some drivers for a training study. The goal of the study is to test whether a driver training program will lead drivers to make better decisions. Drivers in the study are randomly assigned to $X=1$ (recieved optimal driving training) or $X=0$ (recieved basic training that doesn’t encourage optimal descision-making). The outcome variable $Y$ is the amount of revenue the drivers earn in the study period.

Let $Y_{X=1}$ be the revenue earned under exposure to the optimal training and $Y_{X=0}$ be revenue earned under exposure to baseline training. The study showed that the training is highly effected $(E(Y_{X=1}−Y_{X=0})>ϵ)$ where ϵ is some stastical significance threshold.

Your team is debating whether or not you should build that training program into the mobile app. Drivers would opt-in to recieve training and guidance while driving. It would be quite expensive in terms of time and engineering resources to create this app. However, you colleagues say that the expected revenue $E(Y_{X=1}−Y_{X=0})$ would more than make up for the cost.

You argue that most drivers who would opt-in are already highly motivated drivers. You think they would go on to drive more optimally by learning from their own experience, research, seeking out successful drivers, etc.

To demonstrate this, you will estimate the effect of the treatment on the treated (ETT) is $E(Y_{X=1}−Y_{X=0}|X=1)$. In plain English this is the expected difference in earned revenue from those who recieved training relative to what revenue would have been had they not recieved training.

The terms $Y_{X=1}$ and $Y_{X=0}$ in $E(Y_{X=1}−Y_{X=0}|X=1)$ are causal variables, in order to estimate them, you need to convert them into variables than can be estimated directly from data.

The following mathematical derivations show you how to calculate ETT given $Z$, a set of valid adjustment variables that satisfy the backdoor criterion w.r.t $X$ and $Y$.

Firstly, the following is true according to the basic rules of conditional probability.

\begin{align}
P(Y_x=y|X=x′) &= \sum_{z}P(Y_x=y,Z=z|X=x′)\\
&= \sum_{z}P(Y_x=y|Z=z,X=x′)P(Z=z|X=x′)
\end{align}

Secondly, the counterfactual implication of the backdoor criterion is $X\perp Y_x|Z$. This means that $P(Y_x=y|Z=z,X=x′)=P(Y_x=y|Z=z,X=x)$, because conditional on $Z$, the probability of $Y_x$ doesn’t respond to $X$. This leads to the next simplification:

\begin{align}
P(Y_x=y|X=x′) &= \sum_{z}P(Y_x=y,Z=z|X=x′)\\
&= \sum_{z}P(Y_x=y|Z=z,X=x′)P(Z=z|X=x′) \\
&= \sum_{z}P(Y_x=y|Z=z,X=x)P(Z=z|X=x′) \\ 
&= \sum_{z}P(Y=y|Z=z,X=x)P(Z=z|X=x′)
\end{align}

The last step is because covariate adjustment means adjusting for Z allows conditioning on $X = x$ can stand in for $\text{do}(X = x)$.

Hint: All of the terms in the last line are estimable from data. All you need to do is calculate expectations. $x’$ is the counterfactual treatment of regarding to $x$. (i.e. if $X$ is binary, $x = 0$ implies $x’ = 1$, vice versa)

The data for this question is “HW4.csv”. In order to get full credit, show the work of how to calculate $E(Y_{X=0}|X=1)$ , $E(Y_{X=1}−Y_{X=0}|X=1)$, $E(Y_{X=1}−Y_{X=0})$ and then write a short paragraph to explain your results. Use $X$ as training, $Z$ as proxy for motivation, and $Y$ as revenue.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r'D:\Northeastern_Data\Summer_19\CS_7290\HW4.csv')
data = data.iloc[:, 1:]

In [3]:
data.head()

,x,y,z
0,0,1.490090,0
1,1,5.170279,1
2,1,7.434170,1
3,0,1.531446,0
4,0,0.935765,0


In [4]:
# For E[Y_0|X=1]

y_z0 = np.mean(data[(data['x'] == 0) & (data['z'] == 0)]['y'])

y_z1 = np.mean(data[(data['x'] == 0) & (data['z'] == 1)]['y'])

z1 = len(data[(data['x'] == 1) & (data['z'] == 1)])/len(data[data['x'] == 1])

z0 = len(data[(data['x'] == 1) & (data['z'] == 0)])/len(data[data['x'] == 1])

In [5]:
print('E[Y0 | X = 1] = ', (y_z1 * z1) + (y_z0 * z0))

E[Y0 | X = 1] =  3.0909474283549194


In [6]:
print('E[Y | X == 1] = ', str(np.mean(data[data['x'] == 1]['y'])))

E[Y | X == 1] =  4.436555490118973


$ETT = E(Y_{X=1}−Y_{X=0}|X=1) = E[Y | X = 1] - \sum_{z} E[Y|X=0,Z=z]P(Z=z|X=1)$

In [7]:
ETT = ((np.mean(data[data['x'] == 1]['y']))) - ((y_z1 * z1) + (y_z0 * z0))
print('ETT = ', ETT)

ETT =  1.3456080617640538


**We can clearly see that the expected difference in earned revenue from those who recieved training relative to what revenue would have been had they not recieved training is positive. Therefore the training program is beneficial.**

In [8]:
print('E[Y1 -Y0] = ', ETT * (len(data[data['x'] == 1]) / len(data)))

E[Y1 -Y0] =  0.6701128147584988


$E(Y_{X=1}−Y_{X=0}) = E[Y_{X=1}] - E[Y_{X=0}]$ <br>

$E[Y_{X=1}] = E[Y_{X=1}|X=0]P(X=0) + E[Y_{X=1}|X=1]P(X=1)$

In [9]:
# For E[Y_1|X=0]

y_z0 = np.mean(data[(data['x'] == 1) & (data['z'] == 0)]['y'])

y_z1 = np.mean(data[(data['x'] == 1) & (data['z'] == 1)]['y'])

z0 = len(data[(data['x'] == 0) & (data['z'] == 0)])/len(data[data['x'] == 0])

z1 = len(data[(data['x'] == 0) & (data['z'] == 1)])/len(data[data['x'] == 0])

In [10]:
e_y_x_1 = (y_z0*z0 + y_z1*z1)*(len(data[data['x'] == 0])/len(data)) + np.mean(
    data[data['x'] == 1]['y'])*(len(data[data['x'] == 1])/len(data))
print(e_y_x_1)

3.0754568551193047


$E[Y_{X=0}] = E[Y_{X=0}|X=1]P(X=1) + E[Y_{X=0}|X=0]P(X=0)$

In [11]:
# For E[Y_0|X=1]

y_z0 = np.mean(data[(data['x'] == 0) & (data['z'] == 0)]['y'])

y_z1 = np.mean(data[(data['x'] == 0) & (data['z'] == 1)]['y'])

z1 = len(data[(data['x'] == 1) & (data['z'] == 1)])/len(data[data['x'] == 1])

z0 = len(data[(data['x'] == 1) & (data['z'] == 0)])/len(data[data['x'] == 1])

In [12]:
e_y_x_0 = (y_z1 * z1 + y_z0 * z0)*(len(data[data['x'] == 1])/len(data)) + np.mean(
    data[data['x'] == 0]['y'])*(len(data[data['x'] == 0])/len(data))
print(e_y_x_0)

1.861055737142217


So $E(Y_{X=1}−Y_{X=0}) = E[Y_{X=1}] - E[Y_{X=0}]$ <br>

In [13]:
print("E(Y1 - Y0) = ", e_y_x_1 - e_y_x_0)

E(Y1 - Y0) =  1.2144011179770877


**If the cost for the app is less than 1.214, then the app should be built since the expected revenue would more than make up for the cost. This conclusion is drawn from observed data.**